In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
import sys
path = "/content/drive/MyDrive/bm/"
sys.path.append(path)

In [ ]:

import torch
import matplotlib.pyplot as plt
from dbimage_torch import DBImage
plt.rcParams['image.cmap'] = 'Greys_r'
print("Loading MNIST dataset")
with open(path+'data/train_data.pt', 'rb') as f:
    train_X = torch.load(f)
with open(path+'data/train_labels.pt', 'rb') as f:
    train_labels = torch.load(f)
with open(path+'data/test_data.pt', 'rb') as f:
    test_X = torch.load(f)
with open(path+'data/test_labels.pt', 'rb') as f:
    test_labels = torch.load(f)


print("Done loading MNIST")
n_train = 10000
n_test = 1000

test_X = test_X[:n_test,:,:].float()
train_X = train_X[:n_train,:,:].float()
test_labels = test_labels[:n_test].float()
train_labels = train_labels[:n_train].float()

cdns = [1,1,1]

Using CPU
Loading MNIST dataset
Done loading MNIST


In [ ]:
lrs= [1.0,0.5,0.1,0.05,0.01,0.005,0.001]
# lrs = [0.005 ,0.01, 0.05, 0.1]
decays = [0.0001, 0.001, 0.000001]
#moments = [0.0,0.7]

pre_name = path+"runs/MNIST/Pre_training/"
count = 0
for d in decays:
  for l in lrs:
    count += 1
    print(count , " / ", len(decays)*len(lrs))
    name = pre_name + "lr"+str(l) +"_d" + str(d)
    model = DBImage(28,28,10,[500,500,2000],
                        batch_size=20,
                        learning_rate=l,
                        initial_momentum=0.5,
                        momentum=0.9,
                        weight_decay=d,
                        tune_rate=0.0,
                        sparsity_penalty=0,
                        path_writer=name)
    model.set_images(train_X, train_labels,split=0.7)
    model.train(10,save=False, filename=path + "models/db_")
    acc = model.validate(test_X,test_labels,filename=path+"figures/DBN/", full=False, no_print=True, generate=False)



1  /  21




Validating model
2  /  21




Validating model
3  /  21




Validating model
4  /  21




Validating model
5  /  21


...



Validating model
17  /  21




Validating model
18  /  21




Validating model
19  /  21




Validating model
20  /  21




Validating model
21  /  21
